# Loadcase Generator

In [17]:
import pandas as pd
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")

delete_all_previous = True
if delete_all_previous:
    lusas.database().deleteAllAnalyses()
    lusas.database().createAnalysisStructural("Analysis 1", False)
    lusas.database().deleteLoadsets("all") # remaining post-prosessing loadsest

Read the excel definitions

In [18]:
df = pd.read_excel("11 Loadcases.xlsx", sheet_name="Loadcases", usecols=range(0,4))
print(df.head())

                  Name  Count Gravity Analysis
0          Self Weight    NaN     Yes      NaN
1  Self Weight Railing    NaN     NaN      NaN
2            Surfacing    NaN     NaN      NaN
3           Settlement    4.0     NaN      NaN
4            Shrinkage    NaN     NaN      NaN


In [19]:
base_analysis = ''
for analysis in lusas.database().getAnalyses():
    if analysis.isBase():
        base_analysis = analysis.getName()
        break

We'll create a wrapper function that checks for the existance of a loadcase and increments the name

In [20]:
def safe_create_loadcase(loadcase_name:str, analysis_name:str) -> 'IFLoadcase':
    if not lusas.database().existsLoadset(loadcase_name):
        return lusas.database().createLoadcase(loadcase_name, analysis_name)
    else:
        i = 1
        while lusas.database().existsLoadset(f"{loadcase_name}({i})"):
            i+=1
        lusas.database().createLoadcase(f"{loadcase_name} - {i}", analysis_name)

Loop throgh the rows in the loadcase table creating the loadcases as required

In [21]:
for i, row in df.iterrows():
    loadcase_name = row['Name']
    loadcase_count = row['Count']
    # Make sure we have a string for comparison, it should be Yes or empty
    add_gravity = row['Gravity'] if isinstance(row['Gravity'], str) else ""
    # For the analysis name, we need a string and if the analysis doesnt exist we'll need to create it before trying to add the loadcase
    analysis_name = row['Analysis']

    if not isinstance(analysis_name, str):
        analysis_name = base_analysis
    else:
        if not lusas.database().existsAnalysis(analysis_name):
            lusas.database().createAnalysisStructural(analysis_name, False)

    # If the number of requested "similar" loadcases is less than two, create a single loadcase
    if np.isnan(loadcase_count) or int(loadcase_count) < 2:
        loadcase = safe_create_loadcase(loadcase_name, analysis_name)
        if add_gravity == "Yes": loadcase.addGravity(True)
    else:
        # Multiple loadcases
        for i in range(1, int(loadcase_count)+1):
            loadcase = safe_create_loadcase(f"{loadcase_name} {i}", analysis_name)
            if add_gravity == "Yes": loadcase.addGravity(True)


In [22]:
def safe_create_envelope(envelope_name:str) -> 'IFEnvelope':
    if not lusas.database().existsLoadset(envelope_name):
        return lusas.database().createEnvelope(envelope_name)
    else:
        i = 1
        while lusas.database().existsLoadset(f"{envelope_name}({i})"):
            i+=1
        return lusas.database().createEnvelope(f"{envelope_name} - {i}")

Read in the envelope definition sheets and print the top for rows to confirm data as expected

In [23]:
df = pd.read_excel("11 Loadcases.xlsx", sheet_name="Envelopes", usecols=range(0,4))
print(df.head())

                   Name    Loadcases FindSimilar                Folder
0   Settlement Envelope   Settlement         Yes  00 Permanent Actions
1         Wind Envelope         Wind         Yes   01 Variable Actions
2      Thermal Envelope  Temperature         Yes   01 Variable Actions
3  TLO Traffic Envelope    Traffic 1         NaN   01 Variable Actions
4  TLO Traffic Envelope    Traffic 2         NaN   01 Variable Actions


In [24]:


for name in df['Name'].unique():
    #print(name)
    df_env = df[df['Name'] == name]
    #print(df_env)
    # Loadset ids to add to the envelope
    ids = []         
    for i, row in df_env.iterrows():
        loadcase_name = row['Loadcases']
        find_similar = row['FindSimilar']
        # Add all lodsets that start with the given name
        if find_similar == "Yes":
            for loadset in lusas.database().getLoadsets():
                if str(loadset.getName()).startswith(loadcase_name):
                    ids.append(loadset.getID())
        else:
            if lusas.database().existsLoadset(loadcase_name):
                loadset = lusas.database().getLoadset(loadcase_name)
                type_code = loadset.getTypeCode()
                # append the id
                ids.append(loadset.getID())
                if type_code == 3:
                    ids.append(win32.CastTo(loadset, 'IFEnvelope').getAssocLoadset().getID())
                elif type_code == 6:
                    ids.append(win32.CastTo(loadset, 'IFSmarCombination').getAssocLoadset().getID())

    
    env = safe_create_envelope(name)
    for id in ids:
        env.addEntry(id)

    # Set the treeview folder name if available
    folder_name = row['Folder']
    if isinstance(folder_name, str) and len(folder_name) > 0:
        env.setTreeLocation(folder_name, True)
